In [27]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
import pickle
import lightgbm
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from lightgbm import LGBMRegressor


In [2]:
building_metadata_df = pd.read_csv('building_metadata.csv')
train_df = pd.read_csv("train.csv")
weather_train_df = pd.read_csv("weather_train.csv")

In [3]:
temp_df = train_df[['building_id']]
temp_df = temp_df.merge(building_metadata_df, on = ['building_id'], how = 'left')
del temp_df['building_id']
train_df = pd.concat([train_df, temp_df], axis=1)

In [4]:
temp_df = train_df[['site_id','timestamp']]
temp_df = temp_df.merge(weather_train_df, on=['site_id','timestamp'], how='left')
del temp_df['site_id'], temp_df['timestamp']
train_df = pd.concat([train_df, temp_df], axis =1)

In [5]:
train_df.to_pickle('train_df.pkl')


del train_df

In [6]:
train_df = pd.read_pickle('train_df.pkl')


In [7]:
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])

In [8]:
train_df["hour"] = train_df["timestamp"].dt.hour
train_df["day"] = train_df["timestamp"].dt.day
train_df["weekend"] = train_df["timestamp"].dt.weekday
train_df["month"] = train_df["timestamp"].dt.month

In [9]:
#converting primary_use object to integer type
le = LabelEncoder()
train_df['primary_use'] = le.fit_transform(train_df['primary_use']).astype(np.int8)

In [10]:
#Dropping timestamp column
train_df =train_df.drop(columns = ['timestamp'], axis=1)

In [11]:
drop_columns = ['building_id','floor_count','precip_depth_1_hr','sea_level_pressure','wind_direction','wind_speed', 'site_id']
train_df = train_df.drop(columns=drop_columns)

In [12]:
train_df['air_temperature']=train_df['air_temperature'].fillna(20)

In [13]:
train_df['year_built']=train_df['year_built'].fillna(1990)

In [14]:
train_df['cloud_coverage']=train_df['cloud_coverage'].fillna(2)

In [15]:
train_df['dew_temperature']=train_df['dew_temperature'].fillna(15.000000)

In [16]:
y = train_df['meter_reading']

In [17]:
X = train_df.drop(columns=['meter_reading'])

In [18]:
X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 555)

In [19]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


In [20]:

LGB = LGBMRegressor()


In [25]:
LGB.fit(X_train, y_train)
    
    #print("LGB-Mean Absolute Error: %0.4f " % (sklearn.metrics.mean_absolute_error(y_test,LGB.predict(X_test)))
    #print("LGB-R2 score: %0.4f " % (sklearn.metrics.r2_score(y_test,LGB.predict(X_test))))


LGBMRegressor()

In [26]:
pickle.dump(LGB, open('energymodel.pkl','wb'))

In [31]:
s3 = boto3.client('s3')
s3.create_bucket(Bucket='energymlmodel')
s3.upload_file('energymodel.pkl','energymlmodel','energymodel.pkl')